### Setup the environment
https://joelmccune.com/anaconda-and-arcgis-pro/
create a file named `enviroment.yml` containing the following text

```
# https://joelmccune.com/anaconda-and-arcgis-pro/
name: arcgis

channels:
  - esri 
  - conda-forge
  - defaults
  
dependencies:
  - arcgis
  - arcpy
  - deep-learning-essentials
  - cookiecutter
  - nodejs
  - python=3.7
```

open command prompt as administrator
navigate to directory where environment.yml is saved

```
> conda env create -f [C:\path\to\your\]environment.yml
> conda activate arcgis
```

updating the environment, edit the enviromnent.yml file then navigate to the directory and enter the following command in the terminal as administrator. 

```
conda env update --prefix arcgis --file environment.yml  --prune
```

### Preprpcessing 

1. Union > Dissolve Drago 500yr floodplains
2. Project Trueheart to Drago, 
3. Union Trueheart and Drago
4. Dissolve Imperv > calculate field Imperv = 1, Feature to Raster

In [1]:
# setup work space
import arcpy
import os
import glob
import tempfile
# Set environment settings
arcpy.env.workspace = "C:/Workspace/VTWetlands/VTWetlands/"
wd = arcpy.env.workspace + "/"
print(os.listdir(wd))
print(arcpy.env.workspace)
outfolder = arcpy.env.workspace + "/outputs/"
print(outfolder)

# random number generator and seed 
arcpy.env.randomGenerator = "99 ACM599"

# set the number of cores to distribute processing to
arcpy.env.parallelProcessingFactor = "50%"

# output coordinate system
template_SRS = r"D:\GeoData\LandLandcov_BaseLC2016\LandLandcov_BaseLC2016.tif"
spatial_ref = arcpy.Describe(template_SRS).spatialReference
arcpy.env.outputCoordinateSystem = spatial_ref 

# global parameters
nstudents = 71

['.backups', '.ipynb_checkpoints', '.pyHistory', 'AssignRandomPoints2KMLs.ipynb', 'backup.aprx', 'ImportLog', 'Index', 'LandCoverClassification_SamplingPlots.gdb', 'LandCoverClassification_SamplingPlots.lyrx', 'LandLandcov_ImpervSrfcs2016', 'Map1.pdf', 'Map1_CDL19.pdf', 'Map1_Imagery.pdf', 'Map_image.pdf', 'Map_wetland.pdf', 'New Notebook.ipynb', 'outputs', 'sites', 'temp', 'VSWI_Wetlands_Advisory_Layer', 'VSWI_Wetlands_Advisory_Layer.zip', 'VSWI_Wetlands_Class_Layer', 'VSWI_Wetlands_Class_Layer.zip', 'VTWetlands.aprx', 'VTWetlands.aprx.xml', 'VTWetlands.gdb', 'VTWetlands.tbx', 'VTWetlands_2021-03-04.aprx', 'Wetland_Restoration_Model_Site_Prioritization_Lake_Champlain_2017', 'Wetland_Restoration_Model_Site_Prioritization_Lake_Champlain_2017.zip']
C:/Workspace/VTWetlands/VTWetlands
C:/Workspace/VTWetlands/VTWetlands/outputs/


In [20]:
import os
print("PYTHONPATH:", os.environ.get('PYTHONPATH'))
print("PATH:", os.environ.get('PATH'))

PYTHONPATH: None
PATH: C:\ProgramData\Anaconda3\envs\arcgis\Library\bin;C:\PROGRAM FILES\ARCGIS\PRO\BIN;C:\ProgramData\Anaconda3;C:\ProgramData\Anaconda3\Library\mingw-w64\bin;C:\ProgramData\Anaconda3\Library\usr\bin;C:\ProgramData\Anaconda3\Library\bin;C:\ProgramData\Anaconda3\Scripts;C:\Program Files\Microsoft MPI\Bin;C:\Rtools\bin;C:\Program Files (x86)\Common Files\Oracle\Java\javapath;C:\Program Files (x86)\Intel\Intel(R) Management Engine Components\iCLS;C:\Program Files\Intel\Intel(R) Management Engine Components\iCLS;C:\Windows\system32;C:\Windows;C:\Windows\System32\Wbem;C:\Windows\System32\WindowsPowerShell\v1.0;C:\Windows\System32\OpenSSH;C:\Program Files (x86)\Intel\Intel(R) Management Engine Components\DAL;C:\Program Files\Intel\Intel(R) Management Engine Components\DAL;C:\Program Files (x86)\Intel\Intel(R) Management Engine Components\IPT;C:\Program Files\Intel\Intel(R) Management Engine Components\IPT;C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common;C:\Program File

In [21]:
# clip RCPP sites to 
infile = "Wetland_Restoration_Model_Site_Prioritization_Lake_Champlain_2017/Wetland_Restoration_Model_Site_Prioritization_Lake_Champlain_2017.shp"
clipfile = r"C:\Workspace\VTWetlands\VTWetlands\VTWetlands.gdb\Floodplain_500yr_drago_Proje"
clipped = outfolder+"RCCP_Floodplain_Clip.shp"
arcpy.analysis.Clip(infile, clipfile, clipped)

<Result 'C:\\Workspace\\VTWetlands\\VTWetlands\\outputs\\RCCP_Floodplain_Clip.shp'>

In [22]:
# dissolve polygons of constraining feature
inpath = clipped
print(inpath)
outfeatures = "Dissolved.shp"
outpath = outfolder + outfeatures
# check if file exists and delete before running script

if os.path.exists(outpath):
    arcpy.management.Delete(outpath)
if os.path.exists(inpath):
    arcpy.Dissolve_management (inpath, outpath)
else: 
    print("input file does not exist")


C:/Workspace/VTWetlands/VTWetlands/outputs/RCCP_Floodplain_Clip.shp


In [23]:
# create 1 random points inside each polygon
outname = "randompoints.shp"
npoints = nstudents * 10
mindistance = 100
test = os.path.exists(outfolder+outname)
if test:
    print(test)
    arcpy.management.Delete(outfolder+outname)

arcpy.management.CreateRandomPoints(out_path=outfolder, 
                                    out_name=outname, 
                                    constraining_feature_class=inpath,             
                                    number_of_points_or_field=1, 
                                    minimum_allowed_distance= mindistance)

True


<Result 'C:/Workspace/VTWetlands/VTWetlands/outputs\\randompoints.shp'>

In [32]:
# create n random points inside one dissovled polygon
outname = "dissolved_randompoints.shp"
npoints = nstudents * 30
if os.path.exists(outfolder+outname):
    print("deleting ",outfolder+outname)
    arcpy.management.Delete(outfolder+outname)
    
random = arcpy.management.CreateRandomPoints(out_path=outfolder, 
                                    out_name=outname, 
                                    constraining_feature_class=outpath,             
                                    number_of_points_or_field=npoints, 
                                    minimum_allowed_distance= mindistance)

codeblock = '''
def concat(x):
    return('r'+str(x))
    '''
expression = "concat(!FID!)"
inTable = random
fieldName = "Name"
arcpy.management.CalculateField(inTable, fieldName, expression, "PYTHON3", 
                                codeblock)

codeblock = '''
def concat(x):
    return('v'+str(x))
    '''
expression = "concat(!FID!)"
inTable = random
fieldName = "Name"
arcpy.management.CalculateField(inTable, fieldName, expression, "PYTHON3", 
                                codeblock)

deleting  C:/Workspace/VTWetlands/VTWetlands/outputs/dissolved_randompoints.shp


<Result 'C:/Workspace/VTWetlands/VTWetlands/outputs\\dissolved_randompoints.shp'>

In [23]:
# test kml tool
inlayer = outfolder + "dissolved_randompoints.shp"
selection = arcpy.management.SelectLayerByAttribute(in_layer_or_view=inlayer,
                                        selection_type="SUBSET_SELECTION", 
                                        where_clause="FID > 10 And FID <= 20")
arcpy.management.CopyFeatures(selection, "temp/temp")
arcpy.management.Delete(selection)
print(selection)
arcpy.conversion.LayerToGPX(layer="temp/temp",
                            out_kmz_file="temp/temp")
arcpy.FeatureClassToShapefile(Input_Features, Output_Folder)

dissolved_randompoints_Layer32


AttributeError: module 'arcpy.conversion' has no attribute 'LayerToGPX'

In [25]:
# assign random points to students

# change number of points per student
start=1
nstudents= 5
stop= start + nstudents
IDmin= 0
k = 5
npoints= 2 * k
FIDmax= npoints+ FIDmin
y=int(npoints/2)
col="FID"
inlayer = outfolder + "dissolved_randompoints.shp"
#inlayer = r"C:\Workspace\VTWetlands\VTWetlands\LandCoverClassification_SamplingPlots.gdb\Placemarks\Points"
files = [None]*nstudents
print(files)
for x in range(start, stop): #for loop
    print("student", x)
    IDmin=y+IDmin
    if x==start :
        IDmin=0 #for the first student FIDmin=0
    IDmax=IDmin+npoints
    strIDmin=str(IDmin)
    strIDmax=str(IDmax) #converting from int to str 
    print(IDmin, IDmax)
    sql=col+" > " +strIDmin+"  And "+col+" <= "+strIDmax

    if x==nstudents : #for the last student wrap to the first student
        IDmax= IDmin + y
        sqllow=col+" > " +"0"+"  And "+col+" <= "+str(y)
        sqlhigh=" OR "+col+" > " +strIDmin+"  And "+col+" <= "+str(IDmax)
        sql=sqllow + sqlhigh
    print(sql)
    # select by attribute 
    selected = arcpy.management.SelectLayerByAttribute(in_layer_or_view=inlayer,
                                        selection_type="SUBSET_SELECTION", 
                                        where_clause=sql)
    #files[x-start] = arcpy.management.CopyFeatures(selected, "temp/randompoints_"+str(x))
    #arcpy.DeleteFeatures_management(selected)
    arcpy.conversion.LayerToKML(layer=selected,
                                out_kmz_file=outfolder + "kmls/randompoints_"+str(x)+".kmz")
    arcpy.management.Delete(selected)
    


NameError: name 'FIDmin' is not defined

In [11]:
# 2021-03-05 working 

import pandas as pd 
import numpy as np
from shapely.geometry import Point
from arcgis.geometry import Geometry
import math


# read in list of selected sites and shuffle values
df = pd.read_csv(wd+"sites/selected_sites_2021-03-04.csv")
roster = pd.read_csv(wd+"sites/roster.csv")
netid = roster.NETID
print(netid)
print(df)

# create new field name for random points

codeblock = '''
def concat(x):
    return('r'+str(x+1))
    '''
expression = "concat(!FID!)"
random = outfolder + "dissolved_randompoints.shp"
inTable = random
fieldName = "Name"
random = arcpy.management.CalculateField(inTable, fieldName, expression, "PYTHON3", 
                                codeblock)

# create new field name for selected points

0      sralger
1     bbalpard
2     nebingha
3     sbradle2
4     lbrown28
        ...   
64    cwalther
65       rwatt
66      emwong
67      szigic
68    mczimmer
Name: NETID, Length: 69, dtype: object
    IDN      ID        lat        lon  sample_date        project
0     1   MuF.1  44.566992 -73.158673   20180606.0  SurfPlots2018
1     2  ThMB.1  43.968930 -73.154875   20180614.0  SurfPlots2018
2     3   PoW.1  43.744040 -73.054515   20180621.0  SurfPlots2018
3     4    Go.1  43.947220 -73.163715   20180625.0  SurfPlots2018
4     5    Ro.1  43.655005 -73.066960   20180628.0  SurfPlots2018
..  ...     ...        ...        ...          ...            ...
67   68  LPG.FN  44.315099 -73.093247          NaN        FFI2021
68   69   LPG.R  44.311853 -73.096554          NaN        FFI2021
69   70   LP.AS  44.315051 -73.101600          NaN        FFI2021
70   71   LP.RN  44.323403 -73.106364          NaN        FFI2021
71   72   LP.RN  44.322274 -73.107976          NaN        FFI2021

[72

In [10]:
codeblock = '''
def concat(x):
    return('v'+str(x))
    '''
expression = "concat(!IDN!)"
selected = wd+"sites/mygeodata_csv_shp/selected_sites_2021-03-04-point.shp"
#newfeature = "copy_selected_sites"
fieldName = "Name"
selected = arcpy.management.CopyFeatures(in_features=selected,
                                         out_feature_class=wd+"VTWetlands.gdb/copy_selected_sites")
fieldList = arcpy.ListFields(selected)  #get a list of fields for each feature class
print([fn.name  for fn in fieldList])
"""for field in fieldList: #loop through each field
    if field.name.lower() == 'name':  #look for the name elev
        selected = arcpy.AlterField_management(selected, field.name, 'siteID')"""
selected = arcpy.management.CalculateField(selected, fieldName, expression, "PYTHON3", 
                                codeblock)

['OBJECTID', 'Shape', 'IDN', 'ID', 'lat', 'lon', 'sample_dat', 'project', 'Name']


In [68]:
IDmin_sample = 0
IDmin_rand = 0

# SELECTED
start = 0
stop = len(df.IDN)
arr = np.arange(start,stop)
print(arr)
np.random.shuffle(arr)
print('shuffled list\n',arr)
arr[1]

for i in range(5):
    # RANDOM
    print ("iteration:",i)
    print ("IDmin_rand:",IDmin_rand)
    print ("IDmin_sample:",IDmin_sample)
    inlayer = random
    nstudents= len(roster[:3])
    z = 1 # the proportion of nonoverlap, 
    npoints= 5
    col = "FID"
    IDmin_rand = random_selection1(inlayer,nstudents,IDmin_rand,z,npoints,col,netid,i)
       
    # SELECTED
    inlayer = selected
    np.random.shuffle(arr)
    # parameters
    nstudents= len(roster[:3])
    z = 0.5 # the proportion of nonoverlap
    npoints= 2
    nstudentsperpoint = 9
    col = "FID"
    IDmin_sample = sample_selection2(inlayer,nstudents,IDmin_sample,z,npoints,col,netid,i,arr)

[ 0  1  2  3 ... 68 69 70 71]
shuffled list
 [52 30 32 66 ...  6 46 29 33]
iteration: 0
IDmin_rand: 0
IDmin_sample: 0
index 0
student sralger
0 5
FID > 0  And FID <= 5


ExecuteError: ERROR 000210: Cannot create output C:/Workspace/VTWetlands/VTWetlands\temp\points_random0_sralger.shp
Failed to execute (CopyFeatures).


In [38]:
def random_selection(inlayer,nstudents,IDmin,z,npoints,col,netid,i):
    y = math.ceil(npoints*z)
    IDmax= npoints+ IDmin
    files = [None]*nstudents
    for x in range(nstudents): #for loop
        if x>0:
            IDmin=y+IDmin
            IDmax=IDmin+npoints
        print("index", x)
        print("student", netid[x])
        print(IDmin, IDmax)
        # set sql string
        sql=col+" > " +str(IDmin)+"  And "+col+" <= "+str(IDmax)
        
        if x==nstudents : #for the last student wrap to the first student
            IDmax= IDmin + y
            sqllow=col+" > " +"0"+"  And "+col+" <= "+str(y)
            sqlhigh=" OR "+col+" > " +str(IDmin)+"  And "+col+" <= "+str(IDmax)
            sql=sqllow + sqlhigh
        print(sql)
        # select by attribute 
        
        selected = arcpy.management.SelectLayerByAttribute(in_layer_or_view=inlayer,
                                        selection_type="SUBSET_SELECTION", 
                                        where_clause=sql)
        fname = "points_random"+str(i)+"_"+netid[x]
        files[x] = arcpy.management.CopyFeatures(selected, "temp/"+fname)
        #arcpy.conversion.LayerToKML(layer=selected,
        #                            out_kmz_file=outfolder + "kmls/"+fname+".kmz")
        arcpy.management.Delete(selected)
    return(IDmin)

def sample_selection(inlayer,nstudents,IDmin,z,npoints,col,netid,i,arr):
    IDmax= IDmin + npoints
    files = [None]*nstudents
    print(files)
    for x in range(nstudents): #for loop
        if x>0:
            IDmin=y+IDmin
            IDmax=IDmin+npoints
        elif x==(nstudents-1):
            IDmin = IDmax - y
            
        if x==(nstudents-1): #for the last student wrap to the first student
            print("last student")
            rands = arr[0:y]
            print(rands)
            print(IDmin, IDmax)
            sqllow=" OR ".join(["%s = %s" % (col,str(r)) for r in rands])
            rands = arr[int(IDmin):int(IDmax)]
            sqlhigh=" OR ".join(["%s = %s" % (col,str(r)) for r in rands])
            sql=sqllow+" OR "+sqlhigh
        else:
            rands = arr[int(IDmin):int(IDmax)]
            sql=" OR ".join(["%s = %s" % (col,str(r)) for r in rands])
        print("index", x)
        print("student", netid[x])
        print(IDmin, IDmax)
        print(sql)
        # select by attribute 
        selected = arcpy.management.SelectLayerByAttribute(in_layer_or_view=inlayer,
                                        selection_type="SUBSET_SELECTION", 
                                        where_clause=sql)
        fname = "points_sample"+str(i)+"_"+netid[x]
        files[x] = arcpy.management.CopyFeatures(selected, "temp/"+fname)
        #arcpy.conversion.LayerToKML(layer=selected,
        #                            out_kmz_file=outfolder + "kmls/"+fname+".kmz")
        arcpy.management.Delete(selected)
    return(IDmin)

In [40]:
def random_selection1(inlayer,nstudents,IDmin,z,npoints,col,netid,i):
    y = math.ceil(npoints*z)
    IDmax= npoints+ IDmin
    files = [None]*nstudents
    for x in range(nstudents): #for loop
        if x>0:
            IDmin=y+IDmin
            IDmax=IDmin+npoints
        print("index", x)
        print("student", netid[x])
        print(IDmin, IDmax)
        # set sql string
        sql=col+" > " +str(IDmin)+"  And "+col+" <= "+str(IDmax)
        
        if x==nstudents : #for the last student wrap to the first student
            IDmax= IDmin + y
            sqllow=col+" > " +"0"+"  And "+col+" <= "+str(y)
            sqlhigh=" OR "+col+" > " +str(IDmin)+"  And "+col+" <= "+str(IDmax)
            sql=sqllow + sqlhigh
        print(sql)
        # select by attribute 
        
        selected = arcpy.management.SelectLayerByAttribute(in_layer_or_view=inlayer,
                                        selection_type="SUBSET_SELECTION", 
                                        where_clause=sql)
        fname = "points_random"+str(i)+"_"+netid[x]
        files[x] = arcpy.management.CopyFeatures(selected, "temp/"+fname)
        #arcpy.conversion.LayerToKML(layer=selected,
        #                            out_kmz_file=outfolder + "kmls/"+fname+".kmz")
        arcpy.management.Delete(selected)
    return(IDmin)

def sample_selection2(inlayer,nstudents,IDmin,z,npoints,col,netid,i,arr):
    IDmax= IDmin + npoints
    files = [None]*nstudents
    print(files)
    for x in range(nstudents): #for loop
        w = math.floor(x/9)
        IDmin = IDmin+w
        IDmax = IDmin+npoints
        rands = arr[IDmin:IDmax]
        sql = " OR ".join(["%s = %s" % (col,str(r)) for r in rands])
        print("index", x)
        print("student", netid[x])
        print(IDmin, IDmax)
        print(sql)
        # select by attribute 
        selected = arcpy.management.SelectLayerByAttribute(in_layer_or_view=inlayer,
                                        selection_type="SUBSET_SELECTION", 
                                        where_clause=sql)
        fname = "points_sample"+str(i)+"_"+netid[x]
        files[x] = arcpy.management.CopyFeatures(selected, "temp/"+fname)
        #arcpy.conversion.LayerToKML(layer=selected,
        #                            out_kmz_file=outfolder + "kmls/"+fname+".kmz")
        arcpy.management.Delete(selected)
    return(IDmin)

In [130]:
# 2021-03-05 working 

import pandas as pd 
import numpy as np
from shapely.geometry import Point
from arcgis.geometry import Geometry
import math

df = pd.read_excel(wd+"sites/selected_sites_2021-03-04.xlsx")
roster = pd.read_csv(wd+"sites/roster.csv")
netid = roster.NETID
print(netid)
print(df)


start = 0
stop = len(df.IDN)
arr = np.arange(start,stop)
print(arr)
np.random.shuffle(arr)
print('shuffled list\n',arr)
arr[1]

# change number of points per student
inlayer = outfolder + "dissolved_randompoints.shp"
nstudents= len(roster[:3])

IDmin = 0
z = .9 # the proportion of nonoverlap, 
npoints= 10
y = math.ceil(npoints*z)
IDmax= npoints+ IDmin
col = "FID"
files = [None]*nstudents
for x in range(nstudents): #for loop
    if x>0:
        IDmin=y+IDmin
        IDmax=IDmin+npoints
    print("index", x)
    print("student", netid[x])
    print(IDmin, IDmax)
    # set sql string
    sql=col+" > " +str(IDmin)+"  And "+col+" <= "+str(IDmax)

    if x==nstudents : #for the last student wrap to the first student
        IDmax= IDmin + y
        sqllow=col+" > " +"0"+"  And "+col+" <= "+str(y)
        sqlhigh=" OR "+col+" > " +str(IDmin)+"  And "+col+" <= "+str(IDmax)
        sql=sqllow + sqlhigh
    print(sql)
    # select by attribute 
      
    selected = arcpy.management.SelectLayerByAttribute(in_layer_or_view=inlayer,
                                        selection_type="SUBSET_SELECTION", 
                                        where_clause=sql)
    fname = "points_random_"+netid[x]
    files[x] = arcpy.management.CopyFeatures(selected, "temp/"+fname)
    #arcpy.conversion.LayerToKML(layer=selected,
    #                            out_kmz_file=outfolder + "kmls/"+fname+".kmz")
    arcpy.management.Delete(selected)
    

# change number of points per student
inlayer = wd+"sites/selected_sites_2021-03-04-point.shp"

# parameters
nstudents= len(roster[:3])
IDmin= 0

z = 0.5 # the proportion of nonoverlap
npoints= 2
y = math.ceil(npoints*z)
IDmax= IDmin + npoints
col = "FID"
files = [None]*nstudents
print(files)
for x in range(nstudents): #for loop
    if x>0:
        IDmin=y+IDmin
        IDmax=IDmin+npoints
    elif x==(nstudents-1):
        IDmin = IDmax - y
    if x==(nstudents-1): #for the last student wrap to the first student
        print("last student")
        rands = arr[0:y]
        print(rands)
        print(IDmin, IDmax)
        sqllow=" OR ".join(["%s = %s" % (col,str(r)) for r in rands])
        rands = arr[int(IDmin):int(IDmax)]
        sqlhigh=" OR ".join(["%s = %s" % (col,str(r)) for r in rands])
        sql=sqllow+" OR "+sqlhigh
    else:
        rands = arr[int(IDmin):int(IDmax)]
        sql=" OR ".join(["%s = %s" % (col,str(r)) for r in rands])
    print("index", x)
    print("student", netid[x])
    print(IDmin, IDmax)
    print(sql)
    # select by attribute 
    selected = arcpy.management.SelectLayerByAttribute(in_layer_or_view=inlayer,
                                        selection_type="SUBSET_SELECTION", 
                                        where_clause=sql)
    fname = "points_sample_"+netid[x]
    files[x] = arcpy.management.CopyFeatures(selected, "temp/"+fname)
    #arcpy.conversion.LayerToKML(layer=selected,
    #                            out_kmz_file=outfolder + "kmls/"+fname+".kmz")
    arcpy.management.Delete(selected)

0      sralger
1     bbalpard
2     nebingha
3     sbradle2
4     lbrown28
        ...   
64    cwalther
65       rwatt
66      emwong
67      szigic
68    mczimmer
Name: NETID, Length: 69, dtype: object
    IDN    name        lat        lon  sample_date        project
0     1   MuF.1  44.566992 -73.158673   20180606.0  SurfPlots2018
1     2  ThMB.1  43.968930 -73.154875   20180614.0  SurfPlots2018
2     3   PoW.1  43.744040 -73.054515   20180621.0  SurfPlots2018
3     4    Go.1  43.947220 -73.163715   20180625.0  SurfPlots2018
4     5    Ro.1  43.655005 -73.066960   20180628.0  SurfPlots2018
..  ...     ...        ...        ...          ...            ...
67   68  LPG.FN  44.315099 -73.093247          NaN        FFI2021
68   69   LPG.R  44.311853 -73.096554          NaN        FFI2021
69   70   LP.AS  44.315051 -73.101600          NaN        FFI2021
70   71   LP.RN  44.323403 -73.106364          NaN        FFI2021
71   72   LP.RN  44.322274 -73.107976          NaN        FFI2021

[72

In [50]:
def globsearchinpath(inpath,globsearch):
    os.chdir(inpath)
    return(glob.glob(globsearch))
x = 0

outpath = wd + "temp/"
files = globsearchinpath(outpath,"points_*"+netid[x]+"*shp")
# Process:  Create a new empty feature class to append shapefiles to
outname = netid[x]
print(files)

emptyFC = arcpy.CreateFeatureclass_management(outpath,
                                              outname,
                                              "POINT", 
                                              files[0])
merge = arcpy.management.Merge(files,emptyFC, "", "ADD_SOURCE_INFO")

['points_random0_sralger.shp', 'points_random1_sralger.shp', 'points_random_sralger.shp', 'points_sample0_sralger.shp', 'points_sample1_sralger.shp', 'points_sample_sralger.shp']


In [1]:
# project data
#indata = outfolder+"randompoints.shp"
#spatial_ref = arcpy.Describe().spatialReference
#arcpy.management.Project(indata, outfolder+"prj_"+"randompoints.shp", spatial_ref)

# outfile names
outnames = ['vtss_hygr',
'lclu16_wet',
'lclu16_shr',
'lclu16_agr',
'rcpp_score',
'vswi_class',
'vswi_advis']

# value_fields
invarnames = [
    'HyGrNum',
    'Class_name',
    'Class_name',
    'Class_name',
    'RCPP_SCORE',
    'CLASS',
    'CLASS']

pathnames = [r'D:\GeoData\VT_Data_-_NRCS_Soil_Survey_Units-shp\VT_Data_-_NRCS_Soil_Survey_Units.shp',
r'D:\GeoData\LandLandcov_Wetlands2016\LandLandcov_Wetlands2016.gdb\Land_Wetlands2016_poly',
r'D:\GeoData\LandLandcov_Shrublands2016\LandLandcov_Shrublands2016.gdb\Land_Shrublands2016_poly',
r'D:\GeoData\LandLandcov_Agriculture2016\LandLandcov_Agriculture2016.gdb\Land_Agriculture2016_poly',
r'D:\GeoData\VSWI\Wetland_Restoration_Model_Site_Prioritization__Lake_Champlain_2017_-shp\Wetland_Restoration_Model_Site_Prioritization__Lake_Champlain_2017_.shp',
r'D:\GeoData\VSWI\VSWI_Wetlands_Class_Layer-shp\VSWI_Wetlands_Class_Layer.shp',
r'D:\GeoData\VSWI\VSWI_Wetlands_Advisory_Layer-shp\VSWI_Wetlands_Advisory_Layer.shp']

In [4]:
for i in range(len(pathnames)):
    print(i)
    in_features=pathnames[i]
    value_field=invarnames[i]
    out_rasterdataset=outfolder+outnames[i]+".tif"
    print(out_rasterdataset)
    try:
        arcpy.conversion.PolygonToRaster(in_features,
                                     value_field,
                                     out_rasterdataset)
                                     #build_rat = "BUILD")
        print("task complete",os.path.exists(out_rasterdataset))
    except:
        print("task failed\n outpath exists:",os.path.exists(out_rasterdataset))

0
C:/Workspace/VTWetlands/VTWetlands/outputs/vtss_hygr.tif
task complete True
1
C:/Workspace/VTWetlands/VTWetlands/outputs/lclu16_wet.tif
task complete True
2
C:/Workspace/VTWetlands/VTWetlands/outputs/lclu16_shr.tif
task complete True
3
C:/Workspace/VTWetlands/VTWetlands/outputs/lclu16_agr.tif
task complete True
4
C:/Workspace/VTWetlands/VTWetlands/outputs/rcpp_score.tif
task complete True
5
C:/Workspace/VTWetlands/VTWetlands/outputs/vswi_class.tif
task complete True
6
C:/Workspace/VTWetlands/VTWetlands/outputs/vswi_advis.tif
task complete True


In [13]:
def globsearchinpath(inpath,globsearch):
    os.chdir(inpath)
    return(glob.glob(globsearch))

# extract values from list of rasters at points
def arcpy_globExtractMultiValues2Points(inpath,globsearch,inpoints):
    os.chdir(inpath)
    inrasters = glob.glob(globsearch)
    print(inrasters)
    return(arcpy.sa.ExtractMultiValuesToPoints(inpoints,inrasters))

In [ ]:
#inKML = 'C:\Workspace\VTWetlands\VTWetlands\sites\LandCoverClassification_SamplingPlots.kmz'
#arcpy.KMLToLayer_conversion(inKML, wd)
#arcpy.management.Project(wd+'LandCoverClassification_SamplingPlots', outfolder+"sites.shp", spatial_ref)
print(wd)
inpoints = wd+'LandCoverClassification_SamplingPlots.gdb/Placemarks/Points'
print(inpoints)
table=arcpy_globExtractMultiValues2Points(inpath="D:/GeoData/Cropscape_VT",
                                    globsearch="*.tif",
                                    inpoints=inpoints)
table=arcpy_globExtractMultiValues2Points(inpath='D:\\GeoData\\LandLandcov_BaseLC2016\\',
                                    globsearch="*.tif",
                                    inpoints=inpoints)
table=arcpy_globExtractMultiValues2Points(inpath=outfolder,
                                    globsearch="*.tif",
                                    inpoints=inpoints)

In [17]:
print(table)

arcpy.conversion.TableToTable(table, 
                              out_path=r"C:\Workspace\VTWetlands\",
                              out_name=r"extracted_sampling_plots.csv)
                                

SyntaxError: EOL while scanning string literal (<string>, line 4)

In [ ]:
def buildWhereClauseFromList(table, field, valueList):
    """Takes a list of values and constructs a SQL WHERE
    clause to select those values within a given field and table."""

    # Add DBMS-specific field delimiters
    fieldDelimited = arcpy.AddFieldDelimiters(arcpy.Describe(table).path, field)

    # Determine field type
    fieldType = arcpy.ListFields(table, field)[0].type

    # Add single-quotes for string field values
    if str(fieldType) == 'String':
        valueList = ["'%s'" % value for value in valueList]

    # Format WHERE clause in the form of an IN statement
    whereClause = "%s IN(%s)" % (fieldDelimited, ', '.join(map(str, valueList)))
    return whereClause

